In [1]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency, gdf_data

In [54]:
places = ["Kreuzberg", "Warsaw"]
places_gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = places_gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]
places_gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((13.36823 52.49334, 13.36917 52.49315...",52.509382,52.482792,13.452930,13.368229,307579131,relation,55765,52.497644,13.411914,"Kreuzberg, Friedrichshain-Kreuzberg, Berlin, G...",boundary,administrative,0.628946
1,"POLYGON ((20.85169 52.20098, 20.85170 52.20084...",52.368153,52.097850,21.271151,20.851688,307858691,relation,336075,52.231958,21.006725,"Warsaw, Masovian Voivodeship, Poland",boundary,administrative,0.846619


In [26]:
import json
keys = []
nodes = []
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    unique_tags_dict = count_tag_frequency(nodes)
    num_unique_values = {k:len(v) for k, v in unique_tags_dict.items()}
    num_unique_values = {
        k: v
        for k, v in sorted(
            num_unique_values.items(), key=lambda item: item[1], reverse=True
        )
    }

unique_tags_dict_sorted = [unique_tags_dict[k] for k in list(num_unique_values.keys())]

unique_tags_dict

{'description': ['Ks-Mehrabschnittssignal als Einfahrsignal in westlicher Richtung',
  'Garne & Kaffee'],
 'note': ['Zusatzanzeiger fehlen noch',
  'schwarz weiß Mastschild',
  'Die Lounge hat täglich von 15:00 bis 24:00 Uhr geöffnet',
  'versetzt zu Node 960944165'],
 'operator': ['DB Netz AG',
  'Bank für Sozialwirtschaft',
  'Deutsche Telekom AG',
  'Q9396',
  'de:Deutsche Telekom',
  'Olla Orient GmbH',
  'RS',
  'Karl Meyer Rohstoffverwertung',
  'Good News Berlin GmbH',
  'Euronet',
  'Q5412010',
  'en:Euronet Worldwide'],
 'railway': ['signal',
  'DE-ESO:ks',
  'light',
  'entry',
  'normal',
  'backward',
  'bridge',
  'DE-ESO:hp0',
  'DE-ESO:ks1',
  'DE-ESO:ks2',
  'DE-ESO:dr:zs1',
  'forward',
  'DE-ESO:zs3',
  'DE-ESO:zs3v',
  'milestone',
  '3.2',
  'tram_stop',
  'joint',
  'possible',
  '101 VS 4',
  'tram_crossing',
  'tram_level_crossing'],
 'ref': ['19 B', '88', '19 A', '308812', '309432', '09060099', '12'],
 'source': ['bing',
  ' survey',
  'survey',
  'local knowled

In [48]:
def search_dict(d, substring):
    search_words = [s.strip() for s in substring.split(",")]
    print(search_words)
    matches = {}
    for s in search_words:
    # Add key value pairs if a substring appears in either key or value. Value is a list of strings. return only the matching string
        for key, value in d.items():
            if s in key:
                matches[key] = value
            else:
                for v in value:
                    if s in v:
                        if key in matches:
                            matches[key].append(v)
                        else:
                            matches[key] = [v]
    return matches

search_dict(unique_tags_dict, "fire, table")

['fire', 'table']


{'emergency': ['fire_hydrant'],
 'fire_hydrant': ['200', 'sidewalk', 'underground'],
 'sport': ['table_tennis']}

In [49]:
len("asdlfkmoadfibno lkansfoinal , lkaosnidfna ")

42

In [ ]:

dict(zip(places_gdf["display_name"], places_gdf["projected_area"]))

In [ ]:
places_gdf[["projected_area", "area_unit"]] = places_gdf.apply(lambda row: gdf_data(row, places_gdf.crs), axis=1)
places_gdf

In [ ]:
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    keys = list(count_tag_frequency(nodes).keys())
keys

In [ ]:
import utm
from pyproj import CRS
import geopandas as gpd
def gdf_data(gdf):
    """Get the area of a polygon
    This method is not directly callable by the LLM"""
    places_dict = {}
    for index, row in gdf.iterrows():
        print(index)
        utm_zone = utm.latlon_to_zone_number(gdf.loc[[index], "lat"].values[0], gdf.loc[[index], "lon"].values[0])
        south = gdf.loc[[index], "lat"].values[0] < 0
        crs = CRS.from_dict({"proj": "utm", "zone": utm_zone, "south": south})
        epsg_code = crs.to_authority()[1]
        unit = list({ai.unit_name for ai in crs.axis_info})[0]
        gdf_projected = gdf.loc[[index],:].to_crs(epsg_code)
        area = gdf_projected.area.values[0]
        places_dict[row["display_name"]] = {"area":area,
                                            "unit":unit}

    return places_dict

In [ ]:
gdf_data(places_gdf)